In [1]:
import scanpy as sc
import ccAFv2

In [10]:

# Load the data from your CSV file
import scanpy as sc

# Path to the directory containing the 10x files
REH = "D:/Halima's Data/Thesis_2/1_GD428_21136_Hu_REH_Parental/outs/filtered_feature_bc_matrix/"
SUP = "D:/Halima's Data/Thesis_2/2_GD444_21136_Hu_Sup_Parental/outs/filtered_feature_bc_matrix/"
new_human_scRNA = "D:/Halima's Data/Thesis_2/RCode/Cell_Cycle_prediction_with_scATAC_Seq/paper1/Review/pbmc_healthy_human/filtered_feature_bc_matrix/"
new_mouse_scRNA = "D:/Halima's Data/Thesis_2/RCode/Cell_Cycle_prediction_with_scATAC_Seq/paper1/Review/10-k-brain-cells_healthy_mouse/filtered_feature_bc_matrix/"
temp = new_mouse_scRNA
adata = sc.read_10x_mtx(temp, var_names='gene_symbols')

adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pp.filter_genes(adata, min_cells=1)  # Keeps only genes that are expressed in at least 1 cell

# Normalization and log transformation
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)



In [11]:
adata

AnnData object with n_obs × n_vars = 11843 × 21080
    obs: 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'
    uns: 'log1p'

In [12]:
adata.var_names = adata.var_names.astype(str)


In [13]:
adata.var_names[:10]


Index(['Xkr4', 'Rp1', 'Sox17', 'Mrpl15', 'Lypla1', 'Gm37988', 'Tcea1', 'Rgs20',
       'Gm16041', 'Atp6v1h'],
      dtype='object')

In [14]:
# Predict cell cycle phases
predicted_labels, probabilities = ccAFv2.predict_labels(adata, species='mouse', gene_id='symbol')




Running ccAFv2:
    Preparing data for classification...
    Marker genes present in this dataset: 787
    Missing marker genes in this dataset: 73
  Predicting cell cycle state probabilities...
371/371 [==============================] - 1s 2ms/step
  Choosing cell cycle state...
Done.


In [15]:
# Add predictions to the AnnData object
adata.obs['ccAFv2'] = predicted_labels

In [16]:
print(adata.obs[['ccAFv2']].head())


                   ccAFv2
AAACCCAAGCAACTCT-1    qG0
AAACCCACACGCGGTT-1     G1
AAACCCACAGCATACT-1     G1
AAACCCACATACCATG-1     G1
AAACCCAGTCGCACAC-1      S


In [17]:
# Save predictions to a CSV file

REH = "1_GD428_21136_Hu_REH_Parental"
SUP = "2_GD444_21136_Hu_Sup_Parental"

new_human_scRNA = "pbmc_healthy_human"
new_mouse_scRNA = "10-k-brain-cells_healthy_mouse"
temp  = new_mouse_scRNA

adata.obs[['ccAFv2']].to_csv(f"D:/Halima's Data/Thesis_2/RCode/Cell_Cycle_prediction_with_scATAC_Seq/paper1/Training_data/ccAFV2_{temp}.csv")
